<a href="https://colab.research.google.com/github/kyliekwann/FinalProject/blob/main/AnalyzeUP_connect_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Setup Spark**

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 2s (109 kB/s)
Reading package lis

In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-17 19:32:15--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.48MB/s    in 0.2s    

2022-06-17 19:32:16 (5.48 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

## **Load Data from S3 into Spark DataFrame**
- Connect to data storage, then extract that data into a DataFrame. We have the mock data (sample_url) in S3 bucket.

- The dataset (df) mimics our expected final database structure.

In [7]:
sample_url = "https://analyzeup-hz.s3.us-west-1.amazonaws.com/Charity_Navigator_US_States_and_Territories.csv"
file_name = "Charity_Navigator_US_States_and_Territories.csv"

In [9]:
from pyspark import SparkFiles

spark.sparkContext.addFile(sample_url)
df = spark.read.csv(SparkFiles.get(file_name), sep=',', header=True, inferSchema=True)
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+---------+-----+--------------------+-----------------------+------------------------------+---------------------------------+---------------------------------+---------------------+------------------------------------------+-------------------------------------+--------------------------------------+------------------------------------------+-------------------------------------------+----------------------+--------------------+-----------------------------+--------------------+--------------+--------------------+--------------------------+--------------------+--------------------------------+-----------------------------+------------------------------+---------------------------+-------------------------------+-----------------------------+------------------------------------+---------------------------------+----------------------------+-----------------------------+------------

In [29]:
# Load in a sql function to use columns
from pyspark.sql.functions import col
df.columns

df.printSchema()

root
 |-- charityNavigatorURL: string (nullable = true)
 |-- mission: string (nullable = true)
 |-- websiteURL: string (nullable = true)
 |-- tagLine: string (nullable = true)
 |-- charityName: string (nullable = true)
 |-- ein: string (nullable = true)
 |-- orgID: string (nullable = true)
 |-- currentRating__score: string (nullable = true)
 |-- currentRating__ratingID: string (nullable = true)
 |-- currentRating__publicationDate: string (nullable = true)
 |-- currentRating__ratingImage__small: string (nullable = true)
 |-- currentRating__ratingImage__large: string (nullable = true)
 |-- currentRating__rating: string (nullable = true)
 |-- currentRating___rapid_links__related__href: string (nullable = true)
 |-- currentRating__financialRating__score: string (nullable = true)
 |-- currentRating__financialRating__rating: string (nullable = true)
 |-- currentRating__accountabilityRating__score: string (nullable = true)
 |-- currentRating__accountabilityRating__rating: string (nullable = t

In [12]:
df_sample = df[["charityName"]]

## **Connect to the AWS RDS instance and write each DataFrame to its table.**
---



In [24]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-analyzeup.c9mmdejuhxq9.us-west-1.rds.amazonaws.com:5432/analyzeup_project"
config = {"user":"postgres",
          "password (DB instance)": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [30]:
# Write table in RDS
df_sample.write.jdbc(url=jdbc_url, table='df_sample_table', mode=mode, properties=config)
df_sample.show()

+--------------------+
|         charityName|
+--------------------+
|      Impact America|
|Four Corners Mini...|
|Mobile Symphony O...|
|       Big Oak Ranch|
|Community Foundat...|
|Tigers Unlimited ...|
|         neverthirst|
|     Mises Institute|
|Jones Valley Teac...|
|   Children's Harbor|
|McWane Science Ce...|
|Red Mountain Thea...|
|Evangelism Missio...|
|      Birmingham Zoo|
|People for Care a...|
|Boy Scouts of Ame...|
|United Way of Cen...|
|Boys & Girls Club...|
|Boy Scouts of Ame...|
|Tukabatchee Area ...|
+--------------------+
only showing top 20 rows



### ** Schema is created with 8 tables. We will keep editing and adding more helpful tables for the model training.

### ** Model is connected here to read from and write into tables from our dataset.

In [33]:
df.write.jdbc(url=jdbc_url, table='complete_table', mode=mode, properties=config)
df.show()

['charityNavigatorURL',
 'mission',
 'websiteURL',
 'tagLine',
 'charityName',
 'ein',
 'orgID',
 'currentRating__score',
 'currentRating__ratingID',
 'currentRating__publicationDate',
 'currentRating__ratingImage__small',
 'currentRating__ratingImage__large',
 'currentRating__rating',
 'currentRating___rapid_links__related__href',
 'currentRating__financialRating__score',
 'currentRating__financialRating__rating',
 'currentRating__accountabilityRating__score',
 'currentRating__accountabilityRating__rating',
 'category__categoryName',
 'category__categoryID',
 'category__charityNavigatorURL',
 'category__image',
 'cause__causeID',
 'cause__causeName',
 'cause__charityNavigatorURL',
 'cause__image',
 'irsClassification__deductibility',
 'irsClassification__subsection',
 'irsClassification__assetAmount',
 'irsClassification__nteeType',
 'irsClassification__incomeAmount',
 'irsClassification__nteeSuffix',
 'irsClassification__filingRequirement',
 'irsClassification__classification',
 'irs

In [37]:
my_df = spark.read.jdbc(url=jdbc_url, table='complete_table', properties= config) 
my_df.limit(10).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+---------+-----+--------------------+-----------------------+------------------------------+---------------------------------+---------------------------------+---------------------+------------------------------------------+-------------------------------------+--------------------------------------+------------------------------------------+-------------------------------------------+----------------------+--------------------+-----------------------------+--------------------+--------------+--------------------+--------------------------+--------------------+--------------------------------+-----------------------------+------------------------------+---------------------------+-------------------------------+-----------------------------+------------------------------------+---------------------------------+----------------------------+-----------------------------+------------

In [43]:
ein_df = spark.read.jdbc(url=jdbc_url, table='ein_table', properties= config) 
name_df = spark.read.jdbc(url=jdbc_url, table='name_table', properties= config) 
income_df = spark.read.jdbc(url=jdbc_url, table='income_table', properties= config) 
publish_date_df = spark.read.jdbc(url=jdbc_url, table='publish_date', properties= config) 
rating_df = spark.read.jdbc(url=jdbc_url, table='rating_table', properties= config) 
state_df = spark.read.jdbc(url=jdbc_url, table='state_table', properties= config) 


In [44]:
print(ein_df.show(5), name_df.show(5))

+-----+---+
|orgid|ein|
+-----+---+
+-----+---+

+-----+------------------------+
|orgid|organization_charityname|
+-----+------------------------+
+-----+------------------------+

None None
